In [34]:
import pandas as pd

In [35]:
df = pd.read_csv('data\lenta_2024-01-01_2024-12-24_T.csv')
df.columns

<>:1: SyntaxWarning: invalid escape sequence '\l'
<>:1: SyntaxWarning: invalid escape sequence '\l'
C:\Users\andrey\AppData\Local\Temp\ipykernel_28752\3970435422.py:1: SyntaxWarning: invalid escape sequence '\l'
  df = pd.read_csv('data\lenta_2024-01-01_2024-12-24_T.csv')


Index(['Unnamed: 0', 'docid', 'url', 'title', 'modified', 'lastmodtime',
       'type', 'domain', 'status', 'part', 'bloc', 'tags', 'image_url',
       'pubdate', 'text', 'rightcol', 'snippet', 'ticket', 'target'],
      dtype='object')

In [36]:
df = df.drop(['Unnamed: 0', 'modified', 'lastmodtime',
       'type', 'domain', 'status', 'part', 'bloc', 'tags', 'image_url',
       'rightcol', 'snippet'], axis=1)


In [37]:
df = df.dropna(subset=['target'])

In [38]:
df.shape

(38, 7)

In [39]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\andrey\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\andrey\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\andrey\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [40]:
import re

def clean_text(text):
    text = text.replace('\xa0', ' ').replace('\n', ' ')
    text = re.sub(r'[\s]+', ' ', text)
    return re.sub(r'[^\w\s]', '', text).strip()

def remove_stopwords(text):
    stop_words = set(stopwords.words('russian'))
    words = word_tokenize(text)
    filtered_text = ' '.join([word.lower() for word in words if word.lower() not in stop_words])
    return filtered_text

df['words'] = df['text'].apply(clean_text).apply(remove_stopwords)
df.loc[1, 'words']

'фото maksim konstantinov global look press дмитрий воронин аналитики некоторых российских банков заявили снизившемся впервые четыре года числе мошеннических действий отношении клиентов кредитных организаций 2023 году увеличилась доля успешных атак средний размер похищенного стали активно использоваться новые технологии включая нейросети отмечают опрошенные рбк специалисты абсолютных значениях количество попыток совершения мошеннических операций действительно снижается оценил замначальника департамента защиты информации газпромбанка алексей плешков отметил уводить средства счетов жертвы злоумышленники процентном отношении стали чаще очередь тинькофф банке 2023 году выявили рост отношения попыток мошенничества количеству клиентов 17 процентов заявили наблюдаемом буме возникновения финансовых пирамид лжеброкеров попрежнему остро стоящей проблеме телефонным мошенничеством лаборатории касперского также видят снижения числа мошеннических атак банковских клиентов числе количество фишинговых 

In [41]:
from sklearn.model_selection import train_test_split, GridSearchCV

X = df['words']
y = df['target']

In [42]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(max_features=500)
X_vectorized = vectorizer.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_vectorized, y, test_size=0.2, random_state=42)



In [43]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor

param_grids = {
    'LinearRegression': {},  # Нет гиперпараметров для настройки
    'RandomForest': {
        'n_estimators': [100, 200, 300],
        'max_depth': [None, 10, 20],
    },
    'SVR': {
        'kernel': ['linear', 'rbf'],
        'C': [0.1, 1, 10],
        'gamma': ['scale', 'auto'],
    },
}

models = {
    'LinearRegression': LinearRegression(),
    'RandomForest': RandomForestRegressor(random_state=42),
    'SVR': SVR(),
}

In [44]:
metrics = {
    'MSE': mean_squared_error,
    'MAE': mean_absolute_error,
    'R2': r2_score,
}

# Результаты
results = {}

# Обучение моделей
for model_name, model in models.items():
    print(f"Training {model_name}...")

    # Настройка гиперпараметров
    grid_search = GridSearchCV(model, param_grids.get(model_name, {}), cv=3, scoring='neg_mean_squared_error', n_jobs=-1)
    grid_search.fit(X_train, y_train)

    # Лучшие параметры и модель
    best_model = grid_search.best_estimator_
    results[model_name] = {'best_params': grid_search.best_params_}

    # Предсказания
    y_pred = best_model.predict(X_test)

    # Оценка метрик
    results[model_name]['metrics'] = {metric_name: metric_func(y_test, y_pred) for metric_name, metric_func in metrics.items()}
    print(f"Completed {model_name}: {results[model_name]['metrics']}")

# Вывод результатов
print("\nSummary of Results:")
for model_name, result in results.items():
    print(f"{model_name}: Best Params: {result['best_params']} - Metrics: {result['metrics']}")


Training LinearRegression...
Completed LinearRegression: {'MSE': 0.00041562982266398927, 'MAE': 0.01578416426326483, 'R2': -0.10097007810892333}
Training RandomForest...
Completed RandomForest: {'MSE': 0.00038061942994301696, 'MAE': 0.012596016009067592, 'R2': -0.008230354665655248}
Training SVR...
Completed SVR: {'MSE': 0.0004224649060080401, 'MAE': 0.017161830360461025, 'R2': -0.11907566590083762}

Summary of Results:
LinearRegression: Best Params: {} - Metrics: {'MSE': 0.00041562982266398927, 'MAE': 0.01578416426326483, 'R2': -0.10097007810892333}
RandomForest: Best Params: {'max_depth': None, 'n_estimators': 300} - Metrics: {'MSE': 0.00038061942994301696, 'MAE': 0.012596016009067592, 'R2': -0.008230354665655248}
SVR: Best Params: {'C': 0.1, 'gamma': 'scale', 'kernel': 'linear'} - Metrics: {'MSE': 0.0004224649060080401, 'MAE': 0.017161830360461025, 'R2': -0.11907566590083762}
